## A. Window Functions

In MySQL, **window functions** allow you to perform calculations across a set of table rows that are somehow related to the current row. Unlike regular aggregate functions, window functions do not cause rows to become grouped into a single output row; instead, rows retain their separate identities, and the aggregate function is applied over the window of rows. Window functions are often used in scenarios involving ranking, running totals, moving averages, etc.

A **window** in this context is a group of rows defined by the `OVER()` clause, which controls how rows are partitioned and ordered for the window function.

### Common Window Functions in MySQL

1. **`ROW_NUMBER()`**: Assigns a unique sequential integer to rows within a partition of a result set.
2. **`RANK()`**: Similar to `ROW_NUMBER()`, but rows with equal values in the ORDER BY clause receive the same rank, and gaps may appear.
3. **`DENSE_RANK()`**: Like `RANK()`, but it assigns consecutive ranks without gaps.
4. **`NTILE(n)`**: Divides the result set into `n` buckets and assigns a bucket number to each row.
5. **`LAG()`**: Provides access to the value of a row at a given physical offset before the current row.
6. **`LEAD()`**: Provides access to the value of a row at a given physical offset after the current row.
7. **`SUM()`, `AVG()`, `MIN()`, `MAX()`**: Aggregate functions that can be used as window functions with the `OVER()` clause.

### Basic Syntax

```sql
window_function([arguments]) OVER (
    [PARTITION BY partition_expression]
    [ORDER BY order_expression]
    [ROWS or RANGE frame_definition]
)
```

### Example 1: `ROW_NUMBER()`
Let's assign a unique row number to each employee based on their salary.

```sql
SELECT employee_id, first_name, salary,
       ROW_NUMBER() OVER (ORDER BY salary DESC) AS rank
FROM employees;
```

In this query:
- The `ROW_NUMBER()` function assigns a unique number to each row.
- The `ORDER BY salary DESC` orders the rows by salary in descending order.

### Example 2: `RANK()` and `DENSE_RANK()`
Let's rank employees based on their salary, and handle cases where multiple employees have the same salary.

```sql
SELECT employee_id, first_name, salary,
       RANK() OVER (ORDER BY salary DESC) AS rank,
       DENSE_RANK() OVER (ORDER BY salary DESC) AS dense_rank
FROM employees;
```

- The `RANK()` function assigns the same rank to employees with the same salary but skips ranks when there are ties.
- The `DENSE_RANK()` function assigns the same rank to employees with the same salary without skipping ranks.

### Example 3: `LAG()` and `LEAD()`
You can use `LAG()` to access a value from the previous row, and `LEAD()` to access a value from the next row.

```sql
SELECT employee_id, first_name, salary,
       LAG(salary, 1) OVER (ORDER BY salary) AS previous_salary,
       LEAD(salary, 1) OVER (ORDER BY salary) AS next_salary
FROM employees;
```

In this example:
- `LAG(salary, 1)` gets the salary from the previous row.
- `LEAD(salary, 1)` gets the salary from the next row.

### Example 4: Running Total with `SUM()`
To calculate a running total of employee salaries:

```sql
SELECT employee_id, first_name, salary,
       SUM(salary) OVER (ORDER BY salary) AS running_total
FROM employees;
```

Here, `SUM(salary)` computes the running total of the salary column as we move down the result set, ordered by `salary`.

### Example 5: Partitioning with `PARTITION BY`
If you want to rank employees within their departments, you can use the `PARTITION BY` clause.

```sql
SELECT department_id, employee_id, first_name, salary,
       ROW_NUMBER() OVER (PARTITION BY department_id ORDER BY salary DESC) AS department_rank
FROM employees;
```

In this query:
- `PARTITION BY department_id` means that the `ROW_NUMBER()` is reset for each department.

---

Window functions are very powerful for analytical queries, and using them effectively can greatly improve performance when compared to traditional queries with subqueries or joins.

## B. `LAG()` and `LEAD()` Functions

**`LAG()`** and **`LEAD()`** are powerful window functions in MySQL that allow you to access data from a previous or subsequent row within the result set, without the need for complex self-joins or subqueries. They are especially useful for performing calculations that involve comparing a row with its preceding or following row, such as computing differences over time, identifying trends, or performing cumulative analyses.

### **Syntax**

```sql
LAG(expression [, offset [, default_value]]) OVER (
    [PARTITION BY partition_expression]
    [ORDER BY order_expression]
)

LEAD(expression [, offset [, default_value]]) OVER (
    [PARTITION BY partition_expression]
    [ORDER BY order_expression]
)
```

- **`expression`**: The column or calculation you want to retrieve from another row.
- **`offset`** *(optional)*: The number of rows backward (`LAG`) or forward (`LEAD`) from the current row. Default is `1`.
- **`default_value`** *(optional)*: The value returned if the offset goes beyond the scope of the partition (e.g., when there is no previous or next row). Default is `NULL`.
- **`OVER` clause**: Defines the window or set of rows to operate over, which can include `PARTITION BY` and `ORDER BY`.

### **How They Work**

- **`LAG()`**: Fetches data from a preceding row relative to the current row.
- **`LEAD()`**: Fetches data from a following row relative to the current row.

These functions do not alter the number of rows returned by the query; instead, they add additional columns to each row based on other rows in the result set.

### **Detailed Examples**

#### **Example 1: Calculating Difference with Previous Row**

Suppose you have a table `sales` with columns `sale_date` and `total_sales`. You want to calculate the daily sales change.

```sql
SELECT
    sale_date,
    total_sales,
    LAG(total_sales, 1) OVER (ORDER BY sale_date) AS previous_day_sales,
    total_sales - LAG(total_sales, 1) OVER (ORDER BY sale_date) AS sales_change
FROM
    sales
ORDER BY
    sale_date;
```

**Explanation:**

- **`LAG(total_sales, 1) OVER (ORDER BY sale_date)`** retrieves the `total_sales` from the previous day.
- **`total_sales - LAG(total_sales, 1) OVER (ORDER BY sale_date)`** computes the difference between the current day's sales and the previous day's sales.

**Result:**

| sale_date | total_sales | previous_day_sales | sales_change |
|-----------|-------------|--------------------|--------------|
| 2023-01-01 | 1000        | NULL               | NULL         |
| 2023-01-02 | 1500        | 1000               | 500          |
| 2023-01-03 | 1300        | 1500               | -200         |

#### **Example 2: Comparing to Next Row with `LEAD()`**

Suppose you want to compare each product's price with the price of the next product in the list.

```sql
SELECT
    product_id,
    price,
    LEAD(price, 1) OVER (ORDER BY price) AS next_price,
    LEAD(price, 1) OVER (ORDER BY price) - price AS price_difference
FROM
    products
ORDER BY
    price;
```

**Explanation:**

- **`LEAD(price, 1) OVER (ORDER BY price)`** retrieves the price of the next product in ascending order.
- **`LEAD(price, 1) OVER (ORDER BY price) - price`** calculates the difference between the next price and the current price.

#### **Example 3: Handling Multiple Partitions**

Suppose you have employee data and want to compare each employee's salary with the previous salary in their department.

```sql
SELECT
    department_id,
    employee_id,
    salary,
    LAG(salary, 1) OVER (
        PARTITION BY department_id
        ORDER BY salary
    ) AS previous_salary_in_dept
FROM
    employees
ORDER BY
    department_id, salary;
```

**Explanation:**

- **`PARTITION BY department_id`** groups the data by department.
- **`ORDER BY salary`** orders employees within each department by salary.
- **`LAG(salary, 1)`** fetches the previous salary within the department.

#### **Example 4: Specifying Offsets and Default Values**

You can specify how many rows back or ahead to look and provide a default value if the row doesn't exist.

```sql
SELECT
    order_id,
    order_date,
    LAG(order_date, 2, '2000-01-01') OVER (ORDER BY order_date) AS order_date_two_back
FROM
    orders
ORDER BY
    order_date;
```

**Explanation:**

- **`LAG(order_date, 2, '2000-01-01')`** looks two rows back. If there isn't a row two steps back, it returns `'2000-01-01'` as the default.

#### **Example 5: Calculating Time Differences**

Suppose you have user login timestamps and want to calculate the time since each user's last login.

```sql
SELECT
    user_id,
    login_time,
    LAG(login_time, 1) OVER (
        PARTITION BY user_id
        ORDER BY login_time
    ) AS last_login_time,
    TIMESTAMPDIFF(
        MINUTE,
        LAG(login_time, 1) OVER (
            PARTITION BY user_id
            ORDER BY login_time
        ),
        login_time
    ) AS minutes_since_last_login
FROM
    user_logins
ORDER BY
    user_id, login_time;
```

**Explanation:**

- **`TIMESTAMPDIFF(MINUTE, start, end)`** calculates the difference in minutes between two timestamps.
- **This provides insights into user activity patterns.**

### **Use Cases**

1. **Time Series Analysis**: Compare current values with previous or next periods (e.g., stock prices, sales figures).
2. **Data Gaps Identification**: Detect missing data points by comparing timestamps.
3. **Change Detection**: Identify when a value changes compared to the previous or next record.
4. **Calculating Differences**: Compute differences in measurements, such as temperature changes.
5. **Sessionization**: Determine the duration of sessions or time between events.

### **Points to Remember**

- **Order of Rows**: The `ORDER BY` clause in the `OVER()` clause determines the sequence of rows for the window function.
- **Partitioning**: Use `PARTITION BY` to group data into partitions where the window function is applied independently.
- **Handling `NULL` Values**: If there is no preceding or following row (e.g., the first or last row), the functions return `NULL` unless a default value is specified.
- **Performance**: Window functions are generally efficient but be cautious with very large datasets and complex window definitions.

### **Advanced Examples**

#### **Example 6: Cumulative Sum Using `LAG()`**

While `LAG()` isn't typically used for cumulative sums, you can create complex calculations by combining it with other functions.

```sql
SELECT
    transaction_date,
    amount,
    amount + IFNULL(
        LAG(amount, 1) OVER (ORDER BY transaction_date),
        0
    ) AS cumulative_amount
FROM
    transactions
ORDER BY
    transaction_date;
```

**Explanation:**

- This example demonstrates adding the current amount to the previous amount to get a running total.

#### **Example 7: Comparing Non-Adjacent Rows**

You can specify an offset greater than 1 to compare non-adjacent rows.

```sql
SELECT
    date,
    temperature,
    LAG(temperature, 7) OVER (ORDER BY date) AS temperature_last_week,
    temperature - LAG(temperature, 7) OVER (ORDER BY date) AS temperature_change_week
FROM
    weather_data
ORDER BY
    date;
```

**Explanation:**

- Compares the temperature on the same day last week to see weekly changes.

### **Practical Tips**

- **Default Values**: Use default values to handle edge cases where there might not be a preceding or following row.
- **Multiple `LAG()` or `LEAD()` Calls**: You can use multiple instances in the same query to compare multiple rows back or ahead.
- **Combining with Other Window Functions**: Mix `LAG()` and `LEAD()` with other window functions like `ROW_NUMBER()`, `RANK()`, or aggregates for more complex analyses.

### **Common Mistakes**

- **Ignoring the `ORDER BY` Clause**: Without proper ordering, the function may not return meaningful results.
- **Forgetting `PARTITION BY` When Needed**: If analyzing data within groups, neglecting to partition can lead to incorrect comparisons across groups.
- **Not Handling `NULL` Values**: Always consider what should happen when there is no preceding or following row.

### **Summary**

- **`LAG()`** and **`LEAD()`** provide a simple and efficient way to access data from other rows relative to the current one.
- They are essential tools for time series analysis, trend identification, and comparative studies.
- By mastering these functions, you can write more concise and performant SQL queries that reveal deeper insights into your data.

---

Feel free to ask if you have any more questions or need further clarification on using `LAG()` and `LEAD()` in your specific context!

## C.  `IFNULL()` and `COALESCE()`

The `IFNULL()` function in MySQL is used to return a specified value if the expression being evaluated is `NULL`. If the expression is not `NULL`, the `IFNULL()` function will return the expression itself. It's a simple and useful way to handle `NULL` values in your queries, especially when you want to replace them with a default value.

### **Syntax**

```sql
IFNULL(expression, replacement_value)
```

- **`expression`**: The value or column that you want to check for `NULL`.
- **`replacement_value`**: The value to return if the expression is `NULL`. 

### **How It Works**

- If the `expression` is `NULL`, the `IFNULL()` function returns the `replacement_value`.
- If the `expression` is **not** `NULL`, the `IFNULL()` function returns the `expression` itself.

### **Example 1: Replacing `NULL` in a Column**

Suppose you have a table called `orders` with a column `discount`. Some orders might not have a discount applied, and the `discount` field could be `NULL`. You can use `IFNULL()` to replace `NULL` values with `0`.

```sql
SELECT order_id, amount, IFNULL(discount, 0) AS discount
FROM orders;
```

In this query:
- If the `discount` column is `NULL` for any row, `IFNULL()` will return `0` instead.

### **Example 2: Using `IFNULL()` in Calculations**

You might want to calculate the total price for each order, including any discount. If the `discount` column is `NULL`, you can substitute it with `0` so that it doesn’t interfere with your calculations.

```sql
SELECT order_id, amount,
       amount - IFNULL(discount, 0) AS total_price
FROM orders;
```

This ensures that if there is no discount (i.e., the value is `NULL`), the total price is calculated as just the `amount` without subtracting anything.

### **Example 3: Handling `NULL` in Concatenation**

When concatenating strings, `NULL` values can lead to unexpected results. For example, concatenating `NULL` with a string results in `NULL`. You can use `IFNULL()` to handle this.

```sql
SELECT first_name, last_name,
       CONCAT(IFNULL(first_name, ''), ' ', IFNULL(last_name, '')) AS full_name
FROM employees;
```

In this example:
- If `first_name` or `last_name` is `NULL`, `IFNULL()` will replace it with an empty string (`''`) so that the concatenation results in a proper full name.

### **Example 4: Combining with Other Functions**

You can combine `IFNULL()` with other SQL functions for more complex queries. For example, calculating an average where some values might be `NULL`:

```sql
SELECT AVG(IFNULL(score, 0)) AS average_score
FROM student_scores;
```

Here:
- Any `NULL` score is treated as `0` in the average calculation.

### **Difference Between `IFNULL()` and `COALESCE()`**

- **`IFNULL()`**: Only takes two arguments: the expression to check and the replacement value if it is `NULL`.
- **`COALESCE()`**: Can take multiple arguments and returns the first non-`NULL` value.

#### Example of `COALESCE()`:

```sql
SELECT COALESCE(first_name, last_name, 'Unknown') AS name
FROM employees;
```

Here:
- `COALESCE()` will return the first non-`NULL` value in the list of arguments. If `first_name` is `NULL`, it checks `last_name`. If both are `NULL`, it returns `'Unknown'`.

### **Conclusion**

- `IFNULL()` is a simple and effective function for handling `NULL` values in MySQL, replacing them with specified defaults.
- It’s commonly used in calculations, concatenations, and any scenario where you want to avoid `NULL` values disrupting your query logic.

Let me know if you need more examples or further explanations!

## D. `COALESCE()` function

The `COALESCE()` function in MySQL is used to return the first non-`NULL` value from a list of arguments. It is more flexible than `IFNULL()` because it can take multiple arguments and evaluates them in the order they are provided, returning the first value that is not `NULL`.

### **Syntax**

```sql
COALESCE(value1, value2, ..., value_n)
```

- **`value1, value2, ..., value_n`**: A list of expressions. `COALESCE()` returns the first non-`NULL` value from the list. If all the values are `NULL`, it returns `NULL`.

### **How It Works**

- `COALESCE()` evaluates each value in the order given.
- It stops and returns the first value that is not `NULL`.
- If all values are `NULL`, it returns `NULL`.

### **Key Differences from `IFNULL()`**

- **`IFNULL()`**: Takes only two arguments. It returns the second argument if the first one is `NULL`.
- **`COALESCE()`**: Can take multiple arguments and returns the first non-`NULL` value in the sequence.

### **Example 1: Basic Usage**

Suppose you have a table `users` with columns `email`, `phone_number`, and `address`. You want to contact users using the first available contact method (email, phone, or address) and show the preferred method of contact.

```sql
SELECT user_id,
       COALESCE(email, phone_number, address, 'No contact info available') AS preferred_contact
FROM users;
```

**Explanation:**
- `COALESCE(email, phone_number, address, 'No contact info available')` will return the first non-`NULL` value from `email`, `phone_number`, or `address`. 
- If all three are `NULL`, it returns `'No contact info available'`.

### **Example 2: Replacing `NULL` with Defaults**

Suppose you have a table `sales` with columns `sale_price` and `discount`. You want to return the discount for a sale, but if the discount is `NULL`, you want to substitute it with a default value of `0`.

```sql
SELECT sale_id,
       sale_price,
       COALESCE(discount, 0) AS discount
FROM sales;
```

**Explanation:**
- If `discount` is `NULL`, `COALESCE()` will return `0` as the default value.

### **Example 3: Combining with Calculations**

You can use `COALESCE()` in calculations where `NULL` values could affect the result.

```sql
SELECT order_id,
       product_price * quantity AS total_price,
       COALESCE(discount, 0) AS effective_discount,
       (product_price * quantity) - COALESCE(discount, 0) AS final_price
FROM orders;
```

**Explanation:**
- If `discount` is `NULL`, it’s replaced with `0`, ensuring that the final price calculation works properly.

### **Example 4: Handling Multiple Values**

Suppose you have a table `customers` with columns `preferred_contact_email`, `preferred_contact_phone`, and `preferred_contact_address`, and you want to determine how to contact each customer based on their preferred method.

```sql
SELECT customer_id,
       COALESCE(preferred_contact_email, preferred_contact_phone, preferred_contact_address, 'No contact info available') AS contact_method
FROM customers;
```

**Explanation:**
- The query will return the first available contact method in the order of preference (email, then phone, then address). If none is available, it will return `'No contact info available'`.

### **Example 5: Using `COALESCE()` for Multiple Fallbacks**

In cases where multiple fields might have data, and you want to prioritize the non-`NULL` values, `COALESCE()` comes in handy.

```sql
SELECT employee_id,
       COALESCE(bonus, commission, salary * 0.05, 100) AS variable_income
FROM employees;
```

**Explanation:**
- The query returns the employee's bonus if available, otherwise their commission, otherwise 5% of their salary, and if all else is `NULL`, it defaults to `100`.

### **Example 6: COALESCE with Date Values**

Suppose you have a table of user actions with columns `last_login`, `last_purchase`, and `last_profile_update`, and you want to determine the most recent activity date for each user.

```sql
SELECT user_id,
       COALESCE(last_login, last_purchase, last_profile_update, '2000-01-01') AS last_activity
FROM user_actions;
```

**Explanation:**
- The query returns the most recent activity date based on the first non-`NULL` value in the columns. If all activity dates are `NULL`, it returns `'2000-01-01'` as the default.

### **Comparison: `COALESCE()` vs `IFNULL()`**

| Feature                    | `COALESCE()`                        | `IFNULL()`                         |
|----------------------------|-------------------------------------|------------------------------------|
| Number of Arguments         | Multiple                            | Two                                |
| Returns First               | First non-`NULL` value in the list  | Second argument if first is `NULL` |
| Flexibility                 | Higher, with more than two options  | Limited to two options             |

### **Advanced Example: COALESCE with Multiple Columns**

If you have a table of employee data where some fields may be `NULL`, you can use `COALESCE()` to find the first non-`NULL` value for each employee’s salary data.

```sql
SELECT employee_id,
       COALESCE(base_salary, adjusted_salary, minimum_salary, 30000) AS salary
FROM employee_salaries;
```

- This ensures that if `base_salary` is `NULL`, it checks `adjusted_salary`. If that is also `NULL`, it checks `minimum_salary`, and if all are `NULL`, it defaults to `30,000`.

### **Conclusion**

- **`COALESCE()`** is a versatile function that returns the first non-`NULL` value from a list of values.
- It's useful in situations where you need to prioritize non-`NULL` values or provide fallback options.
- Unlike `IFNULL()`, which only handles two arguments, `COALESCE()` can handle multiple arguments and gives you more flexibility when working with potentially `NULL` data.



## E.  Examples

The following query will give top two salary in department group order by salary.

```sql
SELECT department, emp_name, salary,ranks
FROM (
    SELECT department, emp_name, salary,
           RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS ranks
    FROM canada.employee
) as ranked_salary
WHERE ranks <= 2
ORDER BY department, salary DESC;


for the derived table. In MySQL, when using a subquery (the query inside parentheses), the subquery must be given an alias (a name) in order to run successfully.

Here’s the corrected query:

```sql
SELECT department_id, employee_id, salary
FROM (
    SELECT department_id, employee_id, salary,
           ROW_NUMBER() OVER (PARTITION BY department_id ORDER BY salary DESC) AS rank
    FROM employees
) AS ranked_salaries
WHERE rank <= 2
ORDER BY department_id, salary DESC;
```

### Explanation:

1. **Alias for Subquery**: The inner query (within parentheses) is named `ranked_salaries`. Without this alias, MySQL will throw an error.
   
2. **Ranking**: The `ROW_NUMBER()` function assigns a unique rank to each employee within their department based on salary in descending order.

3. **Filtering**: The outer query filters the results to include only rows where the rank is less than or equal to 2 (i.e., the top two salaries per department).

### Example Output:

| department_id | employee_id | salary |
|---------------|-------------|--------|
| 1             | 101         | 90000  |
| 1             | 102         | 85000  |
| 2             | 201         | 75000  |
| 2             | 202         | 70000  |



 The frame in a **window function** defines the subset of rows within the partition (or overall result set) that the function should consider for its calculations. In MySQL, you can specify the frame using the `ROWS` or `RANGE` clauses. A common use case of window frame is calculating running totals, moving averages, etc.

### **Frame Syntax**:

```sql
window_function() OVER (
    [PARTITION BY partition_expression]
    [ORDER BY order_expression]
    [ROWS or RANGE frame_clause]
)
```

The `frame_clause` defines the window frame and can be:

- `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`: Includes all rows from the start of the partition up to the current row.
- `ROWS BETWEEN 1 PRECEDING AND CURRENT ROW`: Includes only the current row and the row immediately before it.
- `ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING`: Includes the current row and all subsequent rows.

Let’s work through an example based on a common use case: calculating **running totals** using a frame.

### **Example Table (Sales Data)**

Let’s assume we have the following `sales` table:

| sale_id | sale_date   | amount |
|---------|-------------|--------|
| 1       | 2024-01-01  | 100    |
| 2       | 2024-01-02  | 150    |
| 3       | 2024-01-03  | 200    |
| 4       | 2024-01-04  | 120    |
| 5       | 2024-01-05  | 170    |

### **Scenario**: Calculate the **running total** of sales using a window frame

We will calculate the running total of sales using the following window function:

```sql
SELECT sale_id, sale_date, amount,
       SUM(amount) OVER (
           ORDER BY sale_date
           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) AS running_total
FROM sales
ORDER BY sale_date;
```

### **Explanation**:
- **`SUM(amount)`**: Calculates the cumulative sum (running total) of the `amount` column.
- **`OVER (ORDER BY sale_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)`**:
  - **`ORDER BY sale_date`**: Defines the order of the rows in which the calculation is applied.
  - **`ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`**: The frame specifies that the calculation should include all rows from the start of the partition (in this case, the beginning of the dataset) up to the current row.

### **Output**:

| sale_id | sale_date  | amount | running_total |
|---------|------------|--------|---------------|
| 1       | 2024-01-01 | 100    | 100           |
| 2       | 2024-01-02 | 150    | 250           |
| 3       | 2024-01-03 | 200    | 450           |
| 4       | 2024-01-04 | 120    | 570           |
| 5       | 2024-01-05 | 170    | 740           |

In this case, the running total is the sum of all sales up to and including the current row.

### **Example with a Moving Average (Using a Frame of 2 Rows)**

Now, let’s calculate a **3-day moving average** (the current day and the 2 previous days).

```sql
SELECT sale_id, sale_date, amount,
       AVG(amount) OVER (
           ORDER BY sale_date
           ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
       ) AS moving_avg
FROM sales
ORDER BY sale_date;
```

### **Explanation**:
- **`AVG(amount)`**: Computes the average of the sales amount.
- **`ROWS BETWEEN 2 PRECEDING AND CURRENT ROW`**: This frame includes the current row and the two preceding rows to calculate the 3-day moving average.

### **Output**:

| sale_id | sale_date  | amount | moving_avg |
|---------|------------|--------|------------|
| 1       | 2024-01-01 | 100    | 100.00     |
| 2       | 2024-01-02 | 150    | 125.00     |
| 3       | 2024-01-03 | 200    | 150.00     |
| 4       | 2024-01-04 | 120    | 156.67     |
| 5       | 2024-01-05 | 170    | 163.33     |

### **Explanation of the Moving Average**:
- For `sale_id 1`: There is only one row, so the moving average is `100`.
- For `sale_id 2`: There are two rows (`100` and `150`), so the moving average is `(100 + 150) / 2 = 125`.
- For `sale_id 3`: There are three rows (`100`, `150`, and `200`), so the moving average is `(100 + 150 + 200) / 3 = 150`.
- For `sale_id 4`: The moving average considers the current row and the two preceding rows (`150`, `200`, `120`), so it is `(150 + 200 + 120) / 3 = 156.67`.
- For `sale_id 5`: The moving average for the last three rows is `(200 + 120 + 170) / 3 = 163.33`.

---

### **Different Frame Variations**:

- **`ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`**: This is for running totals or cumulative aggregates, where the function includes all rows from the start to the current row.
- **`ROWS BETWEEN N PRECEDING AND CURRENT ROW`**: This is for moving windows (like moving averages), where the function considers a sliding window of `N` rows before the current row and the current row itself.
- **`ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING`**: This can be used for calculating aggregates over rows from the current row to the end of the partition.

These frames are highly useful when dealing with cumulative calculations, rolling metrics, and trends over time.





The **"current row"** in a window function is **moving** through the result set, and each row is processed one at a time as the **current row** during query execution.

### How It Works:

- When MySQL processes a query with a window function, it **evaluates each row** in the result set individually.
- For each row, it applies the window function (like `ROW_NUMBER()`, `SUM()`, `AVG()`, etc.) based on the **window frame** defined in the query.
- As the query moves through the result set, each row becomes the **current row**, and the window function performs its calculation using that row and any other rows defined in the window frame.

### Example Breakdown:

Given this query:

```sql
SELECT sale_id, sale_date, amount,
       SUM(amount) OVER (
           ORDER BY sale_date
           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) AS running_total
FROM sales
ORDER BY sale_date;
```

Here’s how it works:

1. **First Row (`sale_id = 1`)**:  
   - The first row (`sale_id = 1`) is the **current row**.
   - The `SUM()` window function calculates the sum from the beginning of the dataset to the current row (just the first row in this case).
   - **Result for current row (`sale_id = 1`)**: Running total is `100`.

2. **Second Row (`sale_id = 2`)**:  
   - Now the second row (`sale_id = 2`) becomes the **current row**.
   - The `SUM()` function calculates the sum from the start of the dataset to the second row.
   - **Result for current row (`sale_id = 2`)**: Running total is `100 + 150 = 250`.

3. **Third Row (`sale_id = 3`)**:  
   - The third row (`sale_id = 3`) becomes the **current row**.
   - The `SUM()` function calculates the sum from the start to the third row.
   - **Result for current row (`sale_id = 3`)**: Running total is `100 + 150 + 200 = 450`.

The **current row** moves with each step in the dataset, and the window function is recalculated based on the row that is currently being processed.

### Visualizing the Moving Current Row

Let’s visualize it with the dataset:

| sale_id | sale_date   | amount | running_total (current row result) |
|---------|-------------|--------|-----------------------------------|
| 1       | 2024-01-01  | 100    | 100                               |
| 2       | 2024-01-02  | 150    | 250                               |
| 3       | 2024-01-03  | 200    | 450                               |
| 4       | 2024-01-04  | 120    | 570                               |
| 5       | 2024-01-05  | 170    | 740                               |

- Each time the **current row** moves to the next row, the window function re-evaluates based on the defined window frame (in this case, "from the beginning up to the current row").

### Summary

- The **current row** is dynamic and moves row by row through the result set.
- The window function is applied to each **current row**, considering all rows within the window frame.
- Each time the current row changes, the window function recalculates based on that row and the defined window frame.

This is why window functions are so powerful for tasks like running totals, moving averages, and rankings, as they provide dynamic calculations that adjust based on the current row.


## F. Moving Windows

In SQL, you can specify various options within a moving (or rolling) window using the `ROWS` and `RANGE` clauses. Here’s an overview of the possibilities:

### 1. Moving Window Options Using `ROWS`

The `ROWS` clause is useful when you want to define a window based on a fixed number of rows before or after the current row.

- **`ROWS BETWEEN n PRECEDING AND CURRENT ROW`**: Calculates the window over the `n` rows before the current row up to and including the current row.
  - Example: `ROWS BETWEEN 6 PRECEDING AND CURRENT ROW` gives a seven-row window ending at the current row.
  
- **`ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`**: Uses all rows from the start of the partition up to the current row.
  
- **`ROWS BETWEEN CURRENT ROW AND n FOLLOWING`**: Calculates a window from the current row to the next `n` rows.
  
- **`ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING`**: Includes the current row and all rows following it until the end of the partition.

- **`ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING`**: Includes all rows in the partition (often used to calculate cumulative values).

### 2. Moving Window Options Using `RANGE`

The `RANGE` clause is similar to `ROWS` but typically based on a range of values (useful for date or numeric columns) rather than a specific number of rows.

- **`RANGE BETWEEN INTERVAL n PRECEDING AND CURRENT ROW`**: Defines the window over a specified range of values (e.g., seven days) ending at the current row.
  - Example: If `n` is 6, it defines a 7-day range, including dates from six days prior to the current date up to the current date.

- **`RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`**: Uses all values from the start of the partition up to the current row’s value.
  
- **`RANGE BETWEEN CURRENT ROW AND INTERVAL n FOLLOWING`**: Defines the range as values from the current row's value to `n` values after the current row.
  
- **`RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING`**: Includes all values within the partition.

### 3. Examples of Use

#### Seven-Day Moving Average Based on Rows

```sql
SELECT
    date,
    AVG(amount_paid) OVER (
        ORDER BY date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS seven_day_moving_avg
FROM payments
ORDER BY date;
```

#### Cumulative Sum

```sql
SELECT
    date,
    SUM(amount_paid) OVER (
        ORDER BY date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS cumulative_sum
FROM payments
ORDER BY date;
```

#### Seven-Day Moving Average Based on Date Range (for Dates with Gaps)

```sql
SELECT
    date,
    AVG(amount_paid) OVER (
        ORDER BY date
        RANGE BETWEEN INTERVAL 6 DAY PRECEDING AND CURRENT ROW
    ) AS seven_day_moving_avg
FROM payments
ORDER BY date;
```

These options provide flexibility for moving window calculations depending on your needs, such as fixed row counts or dynamic ranges based on date intervals. Let me know if you’d like more tailored examples!
